<a href="https://colab.research.google.com/github/remijul/tutorial/blob/master/Data_analysis_FAO_STAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FAO Stat : Data Analysis

## Librairies

In [230]:
# Import des librairies
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
#from google.colab import data_table

## Data

In [2]:
population = pd.read_csv("https://raw.githubusercontent.com/remijul/dataset/master/fao_2013/FAOSTAT_2013_population.csv")
animal = pd.read_csv("https://raw.githubusercontent.com/remijul/dataset/master/fao_2013/FAOSTAT_2013_animal.csv")
vegetal = pd.read_csv("https://raw.githubusercontent.com/remijul/dataset/master/fao_2013/FAOSTAT_2013_vegetal.csv")
cereal = pd.read_csv("https://raw.githubusercontent.com/remijul/dataset/master/fao_2013/FAOSTAT_2013_cereal.csv")
sous_nutrition = pd.read_csv("https://raw.githubusercontent.com/remijul/dataset/master/fao_2013/FAOSTAT_2013_sous_alimentation.csv")

## Exploration des données

A réaliser avant de pouvoir faire les études guidées.
L'exploration des données doit permettre :
- de se familiariser avec l'ensemble des dataset
- d'observer que les unités sont relativement différentes et peuvent représenter un piège par la suite
- de constater que la structure de la table alimentation n'est pas propice à l'analyse et l'exploitation (format long au lieu de large)
- ...

## Etude guidée n°1

Dans cette section, il s’agit ici de répondre à une liste de question spécifique.  
Mais au préalable il convient de préparer les dataset pour harmoniser les unités de chaque dataset et colonnes.

### Préparation des dataset
La suite de l'étude consistera à réaliser un certain de nombre de calculs à partir l'ensemble des informations des dataset. Nous allons donc harmoniser les unités de chaque dataset et colonnes. 

#### Dataset population
Ajustement de l'unité de la population en nombre d'habitant.

In [3]:
population.Value = 1000 * population.Value
population.Unit = '1 person'

#### Dataset sous nutrition
Ajustement de l'unité de la population en nombre d'habitant.

In [48]:
sous_nutrition.Unité.unique()

array(['millions'], dtype=object)

In [50]:
sous_nutrition_prep = sous_nutrition
sous_nutrition_prep['Valeur'] *= 1e6
sous_nutrition_prep['Unité'] = '1 person'

#### Dataset `animal` et `vegetal`
Ajustement de certaines unités. Liste des unités existantes :

In [4]:
animal.Unité.unique()

array(['Milliers de tonnes', 'kg', 'Kcal/personne/jour',
       'g/personne/jour'], dtype=object)

Liste des éléments existants :

In [5]:
animal.Élément.unique()

array(['Production', 'Importations - Quantité',
       'Disponibilité intérieure', 'Nourriture',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Semences', 'Pertes', 'Exportations - Quantité',
       'Aliments pour animaux', 'Autres Utilisations',
       'Variation de stock', 'Traitement'], dtype=object)

Création d'une fonction pour corriger automatiquement les unités :  
- `'Milliers de tonnes'` en `'kg'`,
- `'g/personne/jour'` en `'kg/personne/jour'`,
- ainsi que les informations associées dans les colonnes `Valeur` et `Elément`.

In [6]:
def prepare_dataset(df):
  # Pour les unités en 'Milliers de tonnes'
  df.loc[df['Unité'] == 'Milliers de tonnes', 'Valeur'] *= 1e6 #* df.loc[df['Unité'] == 'Milliers de tonnes', 'Valeur']
  df.loc[df['Unité'] == 'Milliers de tonnes', 'Unité'] = 'kg'

  # Pour les unités en 'g/personne/jour'
  df.loc[df['Unité'] == 'g/personne/jour', 'Valeur'] *= 1e-3 #* df.loc[df['Unité'] == 'kg/personne/jour', 'Valeur']
  #df.loc[df['Unité'] == 'g/personne/jour', 'Élément'] = df.loc[df['Unité'] == 'g/personne/jour', 'Élément'].replace('g/personne/jour', 'kg/personne/jour', regex=True, inplace=True)
  df['Élément'].replace('g/personne/jour', 'kg/personne/jour', regex=True, inplace=True)
  df.loc[df['Unité'] == 'g/personne/jour', 'Unité'] = 'kg/personne/jour'

  return(df)

In [7]:
animal_prep = prepare_dataset(animal)
print(animal_prep.Élément.unique())
print(animal_prep.Unité.unique())

['Production' 'Importations - Quantité' 'Disponibilité intérieure'
 'Nourriture' 'Disponibilité alimentaire en quantité (kg/personne/an)'
 'Disponibilité alimentaire (Kcal/personne/jour)'
 'Disponibilité de protéines en quantité (kg/personne/jour)'
 'Disponibilité de matière grasse en quantité (kg/personne/jour)'
 'Semences' 'Pertes' 'Exportations - Quantité' 'Aliments pour animaux'
 'Autres Utilisations' 'Variation de stock' 'Traitement']
['kg' 'Kcal/personne/jour' 'kg/personne/jour']


In [8]:
vegetal_prep = prepare_dataset(vegetal)
print(vegetal_prep.Élément.unique())
print(vegetal_prep.Unité.unique())

['Production' 'Importations - Quantité' 'Variation de stock'
 'Disponibilité intérieure' 'Semences' 'Pertes' 'Nourriture'
 'Disponibilité alimentaire en quantité (kg/personne/an)'
 'Disponibilité alimentaire (Kcal/personne/jour)'
 'Disponibilité de protéines en quantité (kg/personne/jour)'
 'Disponibilité de matière grasse en quantité (kg/personne/jour)'
 'Aliments pour animaux' 'Exportations - Quantité' 'Traitement'
 'Autres Utilisations']
['kg' 'Kcal/personne/jour' 'kg/personne/jour']


### Question1
Calculez le nombre total d’humains sur la planète.  
Critiquez votre résultat. En cas d’anomalie, analysez et effectuer les corrections nécessaires.  
Donnez le résultat de votre calcul pour l'année 2013.

In [9]:
population_mondiale = population.query('Item == "Population"')['Value'].sum()

print(
    'La population mondiale en 2013 : '
    f'{round(population_mondiale / 10**9, 3):,}', "milliards d'habitants",
)

La population mondiale en 2013 : 8.414 milliards d'habitants


Besoin de corriger le dataset car erreur dans population mondiale.

In [10]:
# Les données de la Chine ont été agrégées dans le code pays 351, et donc comptabilisées 2 fois
population[population['Country'].str.contains('China')]

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
32,FBS,Food Balance Sheets,96,"China, Hong Kong SAR",511,Total Population - Both sexes,2501,Population,2013,2013,1 person,7204000,NaN,Official data
33,FBS,Food Balance Sheets,128,"China, Macao SAR",511,Total Population - Both sexes,2501,Population,2013,2013,1 person,566000,NaN,Official data
34,FBS,Food Balance Sheets,41,"China, mainland",511,Total Population - Both sexes,2501,Population,2013,2013,1 person,1385567000,NaN,Official data
35,FBS,Food Balance Sheets,214,"China, Taiwan Province of",511,Total Population - Both sexes,2501,Population,2013,2013,1 person,23330000,NaN,Official data
174,FBS,Food Balance Sheets,351,China,511,Total Population - Both sexes,2501,Population,2013,2013,1 person,1416667000,A,"Aggregate, may include official, semi-official..."


In [11]:
# Vérification de l'hypothèse des données agrégées
somme_agregat = population[population['Country Code'].isin([96,128,41,214])]['Value'].sum()

print(
    'Valeur du code 351 :', '{:,}'.format(int(population[population['Country Code']==351]['Value'].sum())), '\n',
    'Somme des codes 96, 128, 41 et 214 :', '{:,}'.format(somme_agregat)
)

Valeur du code 351 : 1,416,667,000 
 Somme des codes 96, 128, 41 et 214 : 1,416,667,000


In [18]:
# Fonction de suppression de l'agrégat (code pays 351)
def suppression_agregat(df, column):
    df.drop(df[df[column]==351].index, inplace=True)
    
# Suppression de l'agrégat dans toutes les dataframes
suppression_agregat(df=population, column='Country Code')
suppression_agregat(df=animal_prep, column='Code Pays')
suppression_agregat(df=vegetal_prep, column='Code Pays')
suppression_agregat(df=sous_nutrition, column='Code zone')

In [19]:
# Vérification de la correction des données agrégées
somme_agregat_verif = population[population['Country Code'].isin([96,128,41,214])]['Value'].sum()

print(
    'Valeur du code 351 :', '{:,}'.format(int(population[population['Country Code']==351]['Value'].sum())), '\n',
    'Somme des codes 96, 128, 41 et 214 :', '{:,}'.format(somme_agregat_verif)
)

Valeur du code 351 : 0 
 Somme des codes 96, 128, 41 et 214 : 1,416,667,000


In [22]:
population_mondiale_corrigee = population.query('Item == "Population"')['Value'].sum()

print(
    'La population mondiale en 2013 : '
    f'{round(population_mondiale_corrigee / 10**9, 3):,}', "milliards d'habitants",
)

La population mondiale en 2013 : 6.997 milliards d'habitants


### Question 2
Pour les datasets `animal`, `cereal` et `vegetal`, on considère ici les 11 quantités : Disponibilité intérieure, Production, Importations, Variation, Exportations, Nourriture, Aliments, Animaux, Semences, Pertes et Traitement.  
Identifiez les redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici).  
C'est une équation à 3 termes de type (a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]) ) faisant intervenir chacune des 11 quantités données ci dessus.  
Illustrez cette équation avec l'exemple du blé en France.


In [23]:
liste_quantite = ['Disponibilité intérieure', 'Production', 'Importations - Quantité', 'Variation de stock', 'Exportations - Quantité',
                   'Nourriture', 'Aliments pour animaux', 'Semences', 'Pertes', 'Traitement', 'Autres Utilisations']

vegetal_prep.query('Produit in ("Blé")\
              and Pays in ("France")\
              and Élément in @liste_quantite')

,Code Domaine,Domaine,Code Pays,Pays,Code Élément,Élément,Code Produit,Produit,Code Année,Année,Unité,Valeur,Symbole,Description du Symbole
36006,FBS,Bilans Alimentaires,68,France,5511,Production,2511,Blé,2013,2013,kg,3.861400e+10,S,Données standardisées
36007,FBS,Bilans Alimentaires,68,France,5611,Importations - Quantité,2511,Blé,2013,2013,kg,2.055000e+09,S,Données standardisées
36008,FBS,Bilans Alimentaires,68,France,5072,Variation de stock,2511,Blé,2013,2013,kg,1.131000e+09,S,Données standardisées
36009,FBS,Bilans Alimentaires,68,France,5911,Exportations - Quantité,2511,Blé,2013,2013,kg,2.150200e+10,S,Données standardisées
36010,FBS,Bilans Alimentaires,68,France,5301,Disponibilité intérieure,2511,Blé,2013,2013,kg,2.029800e+10,S,Données standardisées
36011,FBS,Bilans Alimentaires,68,France,5521,Aliments pour animaux,2511,Blé,2013,2013,kg,7.822000e+09,S,Données standardisées
36012,FBS,Bilans Alimentaires,68,France,5527,Semences,2511,Blé,2013,2013,kg,7.480000e+08,S,Données standardisées
36013,FBS,Bilans Alimentaires,68,France,5123,Pertes,2511,Blé,2013,2013,kg,3.580000e+08,S,Données standardisées
36014,FBS,Bilans Alimentaires,68,France,5131,Traitement,2511,Blé,2013,2013,kg,1.575000e+09,S,Données standardisées
36015,FBS,Bilans Alimentaires,68,France,5154,Autres Utilisations,2511,Blé,2013,2013,kg,2.824000e+09,S,Données standardisées


La redondance d'informations peut s'exprimer de la manière suivante :
- Disponibilité intérieure = 
- Production + Importations + Variation de stock - Exportations' = 
- Nourriture + Aliments pour animaux + Semences + Pertes + Traitement + Autres Utilisations

Vérification par calcul :

In [24]:
liste_equil_eco_pos = ['Production', 'Importations - Quantité', 'Variation de stock']
liste_equil_eco_neg = ['Exportations - Quantité']
liste_usage = ['Nourriture', 'Aliments pour animaux', 'Semences', 'Pertes', 'Traitement', 'Autres Utilisations']                   

print(
    '''Blé - France : Disponibilité intérieure = ''' 
    f'''{round(vegetal_prep.query('Produit in ("Blé") and Pays in ("France") and Élément in "Disponibilité intérieure"').Valeur.sum() / 1e3)}''', "Tonnes",
    '\n'
    '''Blé - France : Equilibre économique : ''' 
    f'''{round(vegetal_prep.query('Produit in ("Blé") and Pays in ("France") and Élément in @liste_equil_eco_pos').Valeur.sum()/1e3 -
               vegetal_prep.query('Produit in ("Blé") and Pays in ("France") and Élément in @liste_equil_eco_neg').Valeur.sum()/1e3)}''', "Tonnes"
    '\n'
    '''Blé - France : Usage = ''' 
    f'''{round(vegetal_prep.query('Produit in ("Blé") and Pays in ("France") and Élément in @liste_usage').Valeur.sum()/ 1e3)}''', "Tonnes"
)

Blé - France : Disponibilité intérieure = 20298000 Tonnes 
Blé - France : Equilibre économique : 20298000 Tonnes
Blé - France : Usage = 20298000 Tonnes


### Question 3
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal par an puis en kg de protéines par an. Vous ferez cela à partir de ces informations :
- Population de chaque pays
- Disponibilité alimentaire donnée pour chaque produit et pour chaque pays en kcal/personne/jour,
- Disponibilité alimentaire en protéines donnée pour chaque produit et pour chaque pays en g/personne/jour.

Pour cette étape vous avez besoin de constituer une seule et même table à partir des tables animaux, végétaux et population. Nommer cette nouvelle table « aliments ».

#### Préparation de la table

In [26]:
# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animal_prep['Origine'] = 'Animale'
vegetal_prep['Origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetal_prep, animal_prep])

# Projection
aliments = aliments[[
    'Pays',
    'Code Pays',
    'Produit',
    'Code Produit',
    'Origine',
    'Élément',
    'Valeur',
    'Année'
]]

# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['Code Pays','Pays','Produit','Code Produit','Année','Origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, population[['Country Code', 'Value']], left_on='Code Pays', right_on='Country Code')

# Suppression de la colonne Country car inutile
aliments.drop('Country Code', axis=1, inplace=True)

# # Renommer les colonnes
aliments.rename(columns={
     'Value' : 'Population'
}, inplace=True)

# Remplacer les Nan
aliments.replace(np.nan, 0, inplace=True)

# Affichage
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')
aliments.head()

Résultat :  22 colonnes,  15605 lignes


,Code Pays,Pays,Produit,Code Produit,Année,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,Population
0,1,Arménie,Abats Comestible,2736,2013,Animale,0.0,3000000.0,14.0,4.59,...,17000000.0,0.0,2000000.0,14000000.0,0.0,15000000.0,0.0,0.0,0.0,2977000
1,1,Arménie,"Agrumes, Autres",2614,2013,Végétale,0.0,0.0,0.0,0.46,...,1000000.0,0.0,1000000.0,1000000.0,0.0,0.0,0.0,0.0,0.0,2977000
2,1,Arménie,"Alcool, non Comestible",2659,2013,Végétale,0.0,0.0,0.0,0.00,...,0.0,3000000.0,1000000.0,0.0,0.0,3000000.0,0.0,0.0,0.0,2977000
3,1,Arménie,Aliments pour enfants,2680,2013,Végétale,0.0,0.0,2.0,0.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2977000
4,1,Arménie,Ananas,2618,2013,Végétale,0.0,0.0,0.0,0.16,...,0.0,0.0,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,2977000


#### Intégration des 2 nouvelles colonnes
- disponibilité alimentaire en kcal par an 
- disponibilité alimentaire en kg de protéines par an

In [27]:
#Ajout des Series "Disponibilités alimentaire en Kcal et kg"
aliments['Disponibilité alimentaire (Kcal/an)'] = aliments['Disponibilité alimentaire (Kcal/personne/jour)'] * 365 * aliments['Population'] 
aliments['Disponibilité alimentaire en quantité (kg/an)'] = aliments['Disponibilité alimentaire en quantité (kg/personne/an)'] * aliments['Population']

#"Disponibilités protéines en Kg"
aliments['Disponibilité de protéines en quantité (kg/an)'] = aliments['Disponibilité de protéines en quantité (kg/personne/jour)'] * 365 * aliments['Population']

aliments.head()

,Code Pays,Pays,Produit,Code Produit,Année,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,Population,Disponibilité alimentaire (Kcal/an),Disponibilité alimentaire en quantité (kg/an),Disponibilité de protéines en quantité (kg/an)
0,1,Arménie,Abats Comestible,2736,2013,Animale,0.0,3000000.0,14.0,4.59,...,14000000.0,0.0,15000000.0,0.0,0.0,0.0,2977000,1.521247e+10,13664430.0,2444861.25
1,1,Arménie,"Agrumes, Autres",2614,2013,Végétale,0.0,0.0,0.0,0.46,...,1000000.0,0.0,0.0,0.0,0.0,0.0,2977000,0.000000e+00,1369420.0,10866.05
2,1,Arménie,"Alcool, non Comestible",2659,2013,Végétale,0.0,0.0,0.0,0.00,...,0.0,0.0,3000000.0,0.0,0.0,0.0,2977000,0.000000e+00,0.0,0.00
3,1,Arménie,Aliments pour enfants,2680,2013,Végétale,0.0,0.0,2.0,0.16,...,0.0,0.0,0.0,0.0,0.0,0.0,2977000,2.173210e+09,476320.0,76062.35
4,1,Arménie,Ananas,2618,2013,Végétale,0.0,0.0,0.0,0.16,...,0.0,0.0,0.0,0.0,0.0,0.0,2977000,0.000000e+00,476320.0,0.00


### Question 4
A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit), calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg.  
Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux
données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

In [28]:
# Ratio énergie/poids (Kcal/kg)
aliments['Ratio énergie/poids (Kcal/kg)'] = aliments['Disponibilité alimentaire (Kcal/an)'] / aliments['Disponibilité alimentaire en quantité (kg/an)']

# Ratio aliments['Ratio proteine (kg prot/kg)'] = aliments['Disponibilité alimentaire (Kcal/an)'] / aliments['Disponibilité alimentaire en quantité (kg/an)']
aliments['Ratio proteine (kg prot/kg)'] = aliments['Disponibilité de protéines en quantité (kg/an)'] / aliments['Disponibilité alimentaire en quantité (kg/an)']

In [29]:
aliments.head()

,Code Pays,Pays,Produit,Code Produit,Année,Origine,Aliments pour animaux,Autres Utilisations,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),...,Production,Semences,Traitement,Variation de stock,Population,Disponibilité alimentaire (Kcal/an),Disponibilité alimentaire en quantité (kg/an),Disponibilité de protéines en quantité (kg/an),Ratio énergie/poids (Kcal/kg),Ratio proteine (kg prot/kg)
0,1,Arménie,Abats Comestible,2736,2013,Animale,0.0,3000000.0,14.0,4.59,...,15000000.0,0.0,0.0,0.0,2977000,1.521247e+10,13664430.0,2444861.25,1113.28976,0.178922
1,1,Arménie,"Agrumes, Autres",2614,2013,Végétale,0.0,0.0,0.0,0.46,...,0.0,0.0,0.0,0.0,2977000,0.000000e+00,1369420.0,10866.05,0.00000,0.007935
2,1,Arménie,"Alcool, non Comestible",2659,2013,Végétale,0.0,0.0,0.0,0.00,...,3000000.0,0.0,0.0,0.0,2977000,0.000000e+00,0.0,0.00,NaN,NaN
3,1,Arménie,Aliments pour enfants,2680,2013,Végétale,0.0,0.0,2.0,0.16,...,0.0,0.0,0.0,0.0,2977000,2.173210e+09,476320.0,76062.35,4562.50000,0.159688
4,1,Arménie,Ananas,2618,2013,Végétale,0.0,0.0,0.0,0.16,...,0.0,0.0,0.0,0.0,2977000,0.000000e+00,476320.0,0.00,0.00000,0.000000


In [30]:
print(
    'Le ratio moyen pour un oeuf est : ' 
    f'''{round(aliments.query('Produit == "Oeufs"')['Ratio énergie/poids (Kcal/kg)'].mean(), 1)}''', "Kcal/kg",
)

Le ratio moyen pour un oeuf est : 1364.8 Kcal/kg


### Question 5
En considérant les aliments végétaux et animaux, citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids. Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [31]:
# 20 aliments les plus caloriques (en moyenne)
aliments.groupby(['Produit']).mean() \
.replace([np.inf, -np.inf], np.nan) \
.dropna(subset=["Ratio énergie/poids (Kcal/kg)"], how="all")['Ratio énergie/poids (Kcal/kg)'] \
.sort_values(ascending=False).head(20)

Produit
Huile de Palme              8713.419149
Huile de Soja               8625.483876
Huile de Son de Riz         8497.969939
Huile de Colza&Moutarde     8368.869607
Huil Plantes Oleif Autr     8246.215771
Huile Graines de Coton      8177.340910
Huile de Palmistes          7885.602997
Huile de Germe de Maïs      7827.167158
Huile d'Olive               7774.263485
Graisses Animales Crue      7579.278844
Huiles de Poissons          7133.945055
Beurre, Ghee                6946.278887
Arachides Decortiquees      5466.324644
Sésame                      5010.932143
Feve de Cacao               4343.034036
Riz (Eq Blanchi)            3640.609092
Huiles de Foie de Poisso    3633.539548
Sucre Eq Brut               3578.891980
Haricots                    3256.453914
Sucre non centrifugé        3254.135701
Name: Ratio énergie/poids (Kcal/kg), dtype: float64

In [32]:
# 20 aliments les plus protéiques (en moyenne)
aliments.groupby(['Produit']).mean() \
.replace([np.inf, -np.inf], np.nan) \
.dropna(subset=['Ratio proteine (kg prot/kg)'], how="all")['Ratio proteine (kg prot/kg)'] \
.sort_values(ascending=False).head(20)

Produit
Arachides Decortiquees      0.247908
Soja                        0.247579
Légumineuses Autres         0.220259
Pois                        0.219535
Haricots                    0.215743
Abats Comestible            0.176309
Viande, Autre               0.174102
Sésame                      0.173375
Plantes Oleiferes, Autre    0.156095
Aliments pour enfants       0.148726
Viande de Bovins            0.144611
Cephalopodes                0.140573
Viande d'Ovins/Caprins      0.137363
Viande de Volailles         0.126200
Poissons Pelagiques         0.121300
Piments                     0.109856
Graines Colza/Moutarde      0.109500
Poissons Marins, Autres     0.109425
Oeufs                       0.106961
Épices, Autres              0.106794
Name: Ratio proteine (kg prot/kg), dtype: float64

### Question 6
Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [33]:
#Ajout d'une colonne 'Disponibilité intérieure Kcal'
aliments['Disponibilité intérieure (Kcal)'] = aliments['Disponibilité intérieure'] * aliments['Ratio énergie/poids (Kcal/kg)']

#Ajout d'une colonne 'Disponibilité intérieure kg proteines' => utile pour question 7 et 8
aliments['Disponibilité intérieure de protéines (kg prot)'] = aliments['Disponibilité intérieure'] * aliments['Ratio proteine (kg prot/kg)']

In [35]:
# Extraction du top 10 des produits/pays les plus disponibles
aliments.query('`Disponibilité intérieure (Kcal)` != inf')[['Pays','Produit','Disponibilité intérieure (Kcal)']].sort_values('Disponibilité intérieure (Kcal)', ascending = False).head(10)

,Pays,Produit,Disponibilité intérieure (Kcal)
14314,États-Unis d'Amérique,Maïs,8.032204e+14
2538,"Chine, continentale",Maïs,6.192022e+14
2565,"Chine, continentale",Riz (Eq Blanchi),5.017612e+14
2498,"Chine, continentale",Blé,3.961114e+14
6110,Inde,Riz (Eq Blanchi),3.432776e+14
6043,Inde,Blé,2.691352e+14
2567,"Chine, continentale",Soja,2.620655e+14
1418,Brésil,"Sucre, canne",2.129825e+14
2578,"Chine, continentale",Viande de Suides,1.866875e+14
1385,Brésil,Maïs,1.711875e+14


In [36]:
disponibilite_interieure_kcal = aliments.query('Origine == "Végétale"').replace([np.inf, -np.inf], np.nan)\
.dropna(subset=["Disponibilité intérieure (Kcal)"], how="all")\
['Disponibilité intérieure (Kcal)'].sum()

print(
    'Dispo. intérieure mondiale des végétaux : '
    f'{round(disponibilite_interieure_kcal / 10**9):,}', 'milliards de kcal',
)

Dispo. intérieure mondiale des végétaux : 12,166,422 milliards de kcal


### Question 7
Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ?  
Donnez les résultats en termes de :
- calories, 
- puis de protéines,
- et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.  
 
Vous aurez besoin ici de rechercher et définir le besoin calorique par personne.

#### Potentiel alimentaire en calorie

In [37]:
# Hypothese besoin calorique par personne et par jour
hypothese_besoin_cal = 2350 #kcal/pers/jour

In [38]:
# Résultats en termes de calories
# Utilisation de la 'disponibilite_interieure_kcal' précédemment trouvée
humain_nourris_veg_kcal = round((disponibilite_interieure_kcal / (hypothese_besoin_cal * 365)), 2)

print(
    "En considérant les besoins caloriques journalier d'un être humain de :", hypothese_besoin_cal, "kcal/jour",
    "\n",
    "Le nombre d'humains pouvant être alimentés par les végétaux est de : "
    f"{round(humain_nourris_veg_kcal / 10**9, 3):,}", "milliards d'individus",
    "\n",
    "Soit une couverture de la population mondiale de : "
    f'{round(humain_nourris_veg_kcal/population_mondiale*100, 1):,}', '%',
)

En considérant les besoins caloriques journalier d'un être humain de : 2350 kcal/jour 
 Le nombre d'humains pouvant être alimentés par les végétaux est de : 14.184 milliards d'individus 
 Soit une couverture de la population mondiale de : 168.6 %


#### Potentiel alimentaire en protéine

In [39]:
# Résultats en termes de protéines
disponibilite_interieure_kg_prot = aliments.query('Origine == "Végétale"').replace([np.inf, -np.inf], np.nan)\
.dropna(subset=['Disponibilité intérieure de protéines (kg prot)'], how="all")\
['Disponibilité intérieure de protéines (kg prot)'].sum()

print(
    'Dispo. intérieure mondiale des végétaux : '
    f'{round(disponibilite_interieure_kg_prot / 10**9):,}', 'milliards de kg de protéines',
)

Dispo. intérieure mondiale des végétaux : 292 milliards de kg de protéines


In [40]:
# Résultats en termes de protéines
# Hypothese besoin protéines par personne et par jour
hypothese_besoin_prot = 50 # 50 g/personne/jour

# Utilisation de la 'disponibilite_interieure_kcal' précédemment trouvée (ramenée en kg/an)
humain_nourris_veg_prot = round((disponibilite_interieure_kg_prot / (hypothese_besoin_prot * 1e-3 * 365)), 2)

print(
    "En considérant les besoins journalier de protéines d'un être humain de :", hypothese_besoin_prot, "g de protéines/jour",
    "\n",
    "Le nombre d'humains pouvant être alimentés par les végétaux est de : "
    f"{round(humain_nourris_veg_prot / 10**9, 3):,}", "milliards d'individus",
    "\n",
    "Soit une couverture de la population mondiale de : "
    f'{round(humain_nourris_veg_prot/population_mondiale*100, 1):,}', '%',
)

En considérant les besoins journalier de protéines d'un être humain de : 50 g de protéines/jour 
 Le nombre d'humains pouvant être alimentés par les végétaux est de : 16.0 milliards d'individus 
 Soit une couverture de la population mondiale de : 190.2 %


### Question 8
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux, la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ?  
Donnez les résultats en termes de :
- calories, 
- puis de protéines, 
- et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [41]:
# Ajout des colonnes nécessaires
# en termes de calories
aliments['Nourriture (Kcal)'] = aliments['Nourriture'] * aliments['Ratio énergie/poids (Kcal/kg)']
aliments['Pertes (Kcal)'] = aliments['Pertes'] * aliments['Ratio énergie/poids (Kcal/kg)']

# en termes de protéines
aliments['Nourriture (kg prot)'] = aliments['Nourriture'] * aliments['Ratio proteine (kg prot/kg)']
aliments['Pertes (kg prot)'] = aliments['Pertes'] * aliments['Ratio proteine (kg prot/kg)']

#### Potentiel alimentaire en calorie

In [42]:
# Résultats en termes de calories
disponibilite_alim_vegetaux_kcal = aliments.query('Origine == "Végétale"').replace([np.inf, -np.inf], np.nan)\
.dropna(subset=['Disponibilité alimentaire (Kcal/an)'], how="all")\
[['Disponibilité alimentaire (Kcal/an)', 'Nourriture (Kcal)', 'Pertes (Kcal)']].sum().sum()

print(
    'Dispo. intérieure mondiale des végétaux : '
    f'{round(disponibilite_alim_vegetaux_kcal / 10**9):,}', 'milliards de kcal',
)

# Utilisation de la 'disponibilite_interieure_kcal' précédemment trouvée
humain_nourris_dispo_alim_veg_kcal = round((disponibilite_alim_vegetaux_kcal / (hypothese_besoin_cal * 365)), 2)

print(
    "En considérant les besoins caloriques journalier d'un être humain de :", hypothese_besoin_cal, "kcal/jour",
    "\n",
    "Le nombre d'humains pouvant être alimentés par les végétaux est de : "
    f"{round(humain_nourris_dispo_alim_veg_kcal / 10**9, 3):,}", "milliards d'individus",
    "\n",
    "Soit une couverture de la population mondiale de : "
    f'{round(humain_nourris_dispo_alim_veg_kcal/population_mondiale*100, 1):,}', '%',
)


Dispo. intérieure mondiale des végétaux : 12,608,203 milliards de kcal
En considérant les besoins caloriques journalier d'un être humain de : 2350 kcal/jour 
 Le nombre d'humains pouvant être alimentés par les végétaux est de : 14.699 milliards d'individus 
 Soit une couverture de la population mondiale de : 174.7 %


#### Potentiel alimentaire en protéine

In [43]:
# Résultats en termes de protéines
disponibilite_alim_vegetaux_prot = aliments.query('Origine == "Végétale"').replace([np.inf, -np.inf], np.nan)\
.dropna(subset=['Disponibilité de protéines en quantité (kg/an)'], how="all")\
[['Disponibilité de protéines en quantité (kg/an)', 'Nourriture (kg prot)', 'Pertes (kg prot)']].sum().sum()

print(
    'Dispo. alimentaire mondiale des végétaux : '
    f'{round(disponibilite_alim_vegetaux_prot / 10**9):,}', 'milliards de kg de protéines',
)

# Utilisation de la 'humain_nourris_dispo_alim_veg_prot' précédemment trouvée
humain_nourris_dispo_alim_veg_prot = round((disponibilite_alim_vegetaux_prot / (hypothese_besoin_prot * 1e-3 * 365)), 2)

print(
    "En considérant les besoins journalier de protéines d'un être humain de :", hypothese_besoin_prot, "kcal/jour",
    "\n",
    "Le nombre d'humains pouvant être alimentés par les végétaux est de : "
    f"{round(humain_nourris_dispo_alim_veg_prot / 10**9, 3):,}", "milliards d'individus",
    "\n",
    "Soit une couverture de la population mondiale de : "
    f'{round(humain_nourris_dispo_alim_veg_prot/population_mondiale*100, 1):,}', '%',
)

Dispo. alimentaire mondiale des végétaux : 263 milliards de kg de protéines
En considérant les besoins journalier de protéines d'un être humain de : 50 kcal/jour 
 Le nombre d'humains pouvant être alimentés par les végétaux est de : 14.412 milliards d'individus 
 Soit une couverture de la population mondiale de : 171.3 %


### Question 9
Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ?  
Donnez les résultats en termes de :
- calories, 
- puis de protéines,
- et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

#### Potentiel alimentaire en calorie

In [44]:
# Résultats en termes de calories
disponibilite_alim_kcal = aliments.replace([np.inf, -np.inf], np.nan)\
.dropna(subset=['Disponibilité alimentaire (Kcal/an)'], how="all")\
['Disponibilité alimentaire (Kcal/an)'].sum()

print(
    'Dispo. alimentaire mondiale : '
    f'{round(disponibilite_alim_kcal / 10**9):,}', 'milliards de kcal',
)

# Utilisation de la 'disponibilite_interieure_kcal' précédemment trouvée
humain_nourris_dispo_alim_kcal = round((disponibilite_alim_kcal / (hypothese_besoin_cal * 365)), 2)

print(
    "En considérant les besoins caloriques journalier d'un être humain de :", hypothese_besoin_cal, "kcal/jour",
    "\n",
    "Le nombre d'humains pouvant être alimentés par la disponibilité alimentaire mondiale est de : "
    f"{round(humain_nourris_dispo_alim_kcal / 10**9, 3):,}", "milliards d'individus",
    "\n",
    "Soit une couverture de la population mondiale de : "
    f'{round(humain_nourris_dispo_alim_kcal/population_mondiale*100, 1):,}', '%',
)

Dispo. alimentaire mondiale : 7,363,883 milliards de kcal
En considérant les besoins caloriques journalier d'un être humain de : 2350 kcal/jour 
 Le nombre d'humains pouvant être alimentés par la disponibilité alimentaire mondiale est de : 8.585 milliards d'individus 
 Soit une couverture de la population mondiale de : 102.0 %


#### Potentiel alimentaire en protéine

In [45]:
# Résultats en termes de protéines
disponibilite_alim_prot = aliments.replace([np.inf, -np.inf], np.nan)\
.dropna(subset=['Disponibilité de protéines en quantité (kg/an)'], how="all")\
['Disponibilité de protéines en quantité (kg/an)'].sum()

print(
    'Dispo. alimentaire mondiale : '
    f'{round(disponibilite_alim_prot / 10**9):,}', 'milliards de kg de protéines',
)

# Utilisation de la 'humain_nourris_dispo_alim_veg_prot' précédemment trouvée
humain_nourris_dispo_alim_prot = round((disponibilite_alim_prot / (hypothese_besoin_prot * 1e-3 * 365)), 2)

print(
    "En considérant les besoins journalier de protéines d'un être humain de :", hypothese_besoin_prot, "kcal/jour",
    "\n",
    "Le nombre d'humains pouvant être alimentés par la disponibilité alimentaire mondiale est de : "
    f"{round(humain_nourris_dispo_alim_prot / 10**9, 3):,}", "milliards d'individus",
    "\n",
    "Soit une couverture de la population mondiale de : "
    f'{round(humain_nourris_dispo_alim_prot/population_mondiale*100, 1):,}', '%',
)

Dispo. alimentaire mondiale : 207 milliards de kg de protéines
En considérant les besoins journalier de protéines d'un être humain de : 50 kcal/jour 
 Le nombre d'humains pouvant être alimentés par la disponibilité alimentaire mondiale est de : 11.357 milliards d'individus 
 Soit une couverture de la population mondiale de : 135.0 %


### Question 10
A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [52]:
population_sous_nutrition = sous_nutrition_prep.query('Produit in ("Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)") and\
                                                      Année in ("2012-2014")').Valeur.sum()

print(
    "Le nombre d'humains considérés comme étant en sous-nutrition est de : "
    f"{round(population_sous_nutrition, 3):,}", "millions d'individus",
    "\n",
    "Soit une proportion de la population population mondiale de : "
    f'{round(population_sous_nutrition/population_mondiale*100, 1):,}', '%',
)                                                     

Le nombre d'humains considérés comme étant en sous-nutrition est de : 739,700,000.0 millions d'individus 
 Soit une proportion de la population population mondiale de : 8.8 %


**Remarque :** Cette estimation est une fourchette basse car
- Certains pays ne sont pas recencés dans le jeu de données,
- Pour des pays recencés, il apparaît certaines fois que l'estimation du nombre de personnes sous-alimentées est `Nan` (lorsque que la valeur est par exemple `< 0.1`).

### Question 11
Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.  
En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [110]:
# Liste des céréales
liste_cereales = cereal.Produit.unique()
code_cereales = cereal['Code Produit'].unique()

print("Codes des céréales : ",code_cereales)
print("Liste des céréales : ", liste_cereales)

Codes des céréales :  [2511 2805 2513 2514 2517 2520 2515 2516 2518]
Liste des céréales :  ['Blé' 'Riz (Eq Blanchi)' 'Orge' 'Maïs' 'Millet' 'Céréales, Autres'
 'Seigle' 'Avoine' 'Sorgho']


In [111]:
# Calcul de la proportion
proportion_alim_animale = aliments.query('`Code Produit` in @code_cereales')['Aliments pour animaux'].sum() /\
                      (aliments.query('`Code Produit` in @code_cereales')['Aliments pour animaux'].sum() \
                       + aliments.query('`Code Produit` in @code_cereales')['Nourriture'].sum()
                       )
                      
print(
    "En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), \nla proportion (en termes de poids) destinée à l'alimentation animale est de : "
    f'{round(100 * proportion_alim_animale, 2)}', '%',
)                      

En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), 
la proportion (en termes de poids) destinée à l'alimentation animale est de : 45.91 %


### Question 12
Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition.  
Repérez les 15 produits les plus exportés par ce groupe de pays.  
Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné).  
Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits.  
Ensuite, calculez pour chaque produit les 2 quantités suivantes :
-	le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
-	le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)  

Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)


#### Pays en sous nutrition

In [94]:
pays_sous_nutrition = sous_nutrition_prep.dropna(subset=['Valeur'])[['Code zone', 'Zone']].drop_duplicates()
pays_sous_nutrition.head()

,Code zone,Zone
0,2,Afghanistan
1,202,Afrique du Sud
2,3,Albanie
3,4,Algérie
6,7,Angola


#### Liste des 15 produits les plus exportés par ce groupe de pays

In [80]:
top_15_export = aliments[aliments['Code Pays'].isin(list(pays_sous_nutrition['Code zone']))].sort_values('Exportations - Quantité', ascending=False)[['Produit', 'Pays', 'Exportations - Quantité']].head(15)
top_15_export.head()

,Produit,Pays,Exportations - Quantité
13385,Manioc,Thaïlande,2.521400e+10
6167,Huile de Palme,Indonésie,2.333300e+10
556,Maïs,Argentine,2.024100e+10
8457,Huile de Palme,Malaisie,1.962200e+10
6110,Riz (Eq Blanchi),Inde,1.133700e+10


#### Liste des 200 plus grandes importations 

In [98]:
top_200_import = aliments[aliments['Produit'].isin(list(top_15_export['Produit']))]\
  .sort_values('Importations - Quantité', ascending=False)[['Produit', 'Pays', 'Importations - Quantité', 'Autres Utilisations', 'Disponibilité intérieure', 'Aliments pour animaux', 'Nourriture']].head(200)

top_200_import.head()

,Produit,Pays,Importations - Quantité,Autres Utilisations,Disponibilité intérieure,Aliments pour animaux,Nourriture
2567,Soja,"Chine, continentale",6.338100e+10,0.000000e+00,7.508100e+10,9.530000e+09,5.072000e+09
2537,Manioc,"Chine, continentale",2.904600e+10,7.940000e+09,3.359000e+10,2.286800e+10,2.645000e+09
7015,Maïs,Japon,1.440300e+10,2.880000e+08,1.466100e+10,1.096400e+10,1.171000e+09
3686,Blé,Égypte,1.033100e+10,0.000000e+00,1.934100e+10,4.860000e+09,1.204800e+10
7539,Maïs,République de Corée,8.755000e+09,4.700000e+07,8.749000e+09,6.281000e+09,6.140000e+08


#### Préparation de la table agrégée par produit

In [100]:
table_agr_top_15_export = top_200_import.groupby('Produit')[['Importations - Quantité', 'Autres Utilisations', 'Disponibilité intérieure', 'Aliments pour animaux', 'Nourriture']].sum()
table_agr_top_15_export.head()

,Importations - Quantité,Autres Utilisations,Disponibilité intérieure,Aliments pour animaux,Nourriture
Produit,,,,,
Bananes,1.063800e+10,0.000000e+00,8.441000e+09,0.000000e+00,7.932000e+09
Blé,1.614380e+11,1.821400e+10,4.904870e+11,1.053310e+11,3.213280e+11
Huile de Palme,3.468400e+10,2.293700e+10,3.117200e+10,0.000000e+00,7.971000e+09
"Légumes, Autres",2.903400e+10,1.900000e+07,8.796700e+10,3.980000e+09,7.648100e+10
Manioc,3.549900e+10,2.119100e+10,6.822900e+10,2.515200e+10,1.531800e+10


#### Ratio entre la quantité destinés aux "Autres utilisations" et la disponibilité intérieure

In [102]:
table_agr_top_15_export['Ratio Autres utilisations et Dispo intérieure'] = table_agr_top_15_export['Autres Utilisations'] / table_agr_top_15_export['Disponibilité intérieure']
table_agr_top_15_export.head()


,Importations - Quantité,Autres Utilisations,Disponibilité intérieure,Aliments pour animaux,Nourriture,Ratio Autres utilisations et Dispo intérieure
Produit,,,,,,
Bananes,1.063800e+10,0.000000e+00,8.441000e+09,0.000000e+00,7.932000e+09,0.000000
Blé,1.614380e+11,1.821400e+10,4.904870e+11,1.053310e+11,3.213280e+11,0.037135
Huile de Palme,3.468400e+10,2.293700e+10,3.117200e+10,0.000000e+00,7.971000e+09,0.735821
"Légumes, Autres",2.903400e+10,1.900000e+07,8.796700e+10,3.980000e+09,7.648100e+10,0.000216
Manioc,3.549900e+10,2.119100e+10,6.822900e+10,2.515200e+10,1.531800e+10,0.310586


In [106]:
print(
    "Les 3 produits qui ont la plus grande valeur du ratio Autres utilisations / Dispo intérieure : ",
    table_agr_top_15_export.sort_values('Ratio Autres utilisations et Dispo intérieure', ascending=False)['Ratio Autres utilisations et Dispo intérieure'].head(3)
) 

Les 3 produits qui ont la plus grande valeur du ratio Autres utilisations / Dispo intérieure :  Produit
Huile de Palme    0.735821
Manioc            0.310586
Maïs              0.236274
Name: Ratio Autres utilisations et Dispo intérieure, dtype: float64


#### Ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [107]:
table_agr_top_15_export['Ratio Nourriture animale et Nourriture totale'] = table_agr_top_15_export['Aliments pour animaux'] / table_agr_top_15_export['Nourriture']
table_agr_top_15_export.head()

,Importations - Quantité,Autres Utilisations,Disponibilité intérieure,Aliments pour animaux,Nourriture,Ratio Autres utilisations et Dispo intérieure,Ratio Nourriture animale et Nourriture totale
Produit,,,,,,,
Bananes,1.063800e+10,0.000000e+00,8.441000e+09,0.000000e+00,7.932000e+09,0.000000,0.000000
Blé,1.614380e+11,1.821400e+10,4.904870e+11,1.053310e+11,3.213280e+11,0.037135,0.327799
Huile de Palme,3.468400e+10,2.293700e+10,3.117200e+10,0.000000e+00,7.971000e+09,0.735821,0.000000
"Légumes, Autres",2.903400e+10,1.900000e+07,8.796700e+10,3.980000e+09,7.648100e+10,0.000216,0.052039
Manioc,3.549900e+10,2.119100e+10,6.822900e+10,2.515200e+10,1.531800e+10,0.310586,1.641990


In [108]:
print(
    "Les 3 produits qui ont la plus grande valeur du ratio Nourriture animale / Nourriture totale : ",
    table_agr_top_15_export.sort_values('Ratio Nourriture animale et Nourriture totale', ascending=False)['Ratio Nourriture animale et Nourriture totale'].head(3)
) 

Les 3 produits qui ont la plus grande valeur du ratio Nourriture animale / Nourriture totale :  Produit
Maïs                   6.989189
Poissons Pelagiques    2.286462
Soja                   1.830892
Name: Ratio Nourriture animale et Nourriture totale, dtype: float64


### Question 13
Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [126]:
cereal_usa = aliments.query('''`Code Produit` in @code_cereales and Pays in "États-Unis d'Amérique"''')['Aliments pour animaux'].sum() * 0.1

print(
    "Si les USA diminuaient leur production de produits animaux de 10%, cela libèrerait : "
    f'{round(cereal_usa / 1e6, 2)}', 'Milliers de Tonnes de céréales',
)

Si les USA diminuaient leur production de produits animaux de 10%, cela libèrerait : 14009.6 Milliers de Tonnes de céréales


### Question 14
En Thaïlande, quelle proportion de manioc est exportée ?  
Quelle est la proportion de personnes en sous-nutrition ?

In [155]:
manioc_thai_export = aliments.query('''`Produit` in 'Manioc' and Pays in "Thaïlande"''')['Exportations - Quantité'] / aliments.query('''`Produit` in 'Manioc' and Pays in "Thaïlande"''')['Disponibilité intérieure']

print(
    "En Thaïlande, la proportion de manioc est exportée vis-à-vis de la Disponibilité intérieure est de : "
    f'{(100*manioc_thai_export.values).round(1)[0]}', '%',
)

En Thaïlande, la proportion de manioc est exportée vis-à-vis de la Disponibilité intérieure est de : 402.5 %


In [171]:
prop_sous_alim_thai = sous_nutrition_prep.query('''Zone in "Thaïlande"''')['Valeur'].values / population.query('''Country in "Thailand"''')['Value'].values

print(
    "En Thaïlande, la proportion de la population considérée en sous-alimentation est de : "
    f'{(100*prop_sous_alim_thai).round(1)[0]}', '%',
)

En Thaïlande, la proportion de la population considérée en sous-alimentation est de : 9.1 %




---




## Etude guidée n°2

Il s’agit ici d’intégrer toutes les données dans une base de données relationnelle puis de répondre à toutes les questions ci-dessous.  
La base de données contiendra 4 tables :
- population,
- dispo_alim,
- equilibre_prod,
- sous_nutrition.


### Intégration des données dans une base de données relationnelle
Préparation et configuration de la base.

In [192]:
import sqlite3
con = sqlite3.connect("fao.db")
cur = con.cursor()

### Question 15
Définir une clé primaire pertinente pour la table population.  
Intégrer la table dans la base de données relationnelle.

La clé primaire retenue pour la table population sera le code pays.

In [ ]:
#@title
#Création de la table population
cur.execute('''CREATE TABLE population(
           code_pays INTEGER PRIMARY KEY,
           pays TEXT,
           annee INTEGER,
           population REAL)''')

In [188]:
# Preparation de la table population
population_prep = population[['Country Code', 'Country', 'Year', 'Value']]
population_prep.rename(columns={'Country Code': 'code_pays', 'Country': 'pays', 'Year': 'annee', 'Value': 'population'},inplace=True)

In [190]:
# Alimentation de la bdd
population_prep.to_sql('population', con, if_exists='append', index=False)

In [196]:
# Vérification
cur.execute(""" SELECT * FROM population LIMIT 3 """)
cur.fetchall()

[(1, 'Armenia', 2013, 2977000.0),
 (2, 'Afghanistan', 2013, 30552000.0),
 (3, 'Albania', 2013, 3173000.0)]

### Question 16
La table appelée dispo_alim contient pour chaque pays, pour chaque produit, sur l'année 2013, les informations suivantes:
- la nature du produit (deux valeurs possibles“animal” ou “végétal”)
- disponibilité alimentaire en tonnes
- disponibilité alimentaire en Kcal/personne/jour
- disponibilité alimentaire de protéines en g/personne/jour
- disponibilité alimentaire de matières grasses en g/personne/jour

La table contient ces colonnes :
- pays,
- code_pays,
- année,
- produit,
- code_produit,
- origin,
- dispo_alim_tonnes,
- dispo_alim_kcal_p_j,
- dispo_prot,
- dispo_mat_gr.

Définir une clé primaire pertinente pour la table dispo_alim.  
Intégrer la table dans la base de données relationnelle.


La clé primaire de la table dispo_alim sera id (clé artificielle).  
En cas d'évolution future, la création de l'id [PK] est une solution qui restera stable.

In [209]:
cur.execute('''DROP TABLE dispo_alim''')

In [210]:
#Création de la table population
cur.execute('''CREATE TABLE dispo_alim(
            id INTEGER primary key,
            code_pays INTEGER,
            pays TEXT,
            annee INTEGER,
            produit TEXT,
            code_produit INTEGER,
            origin TEXT,
            dispo_alim_tonnes REAL,
            dispo_alim_kcal_p_j REAL,
            dispo_prot REAL,
            dispo_mat_gr REAL)''')

In [211]:
#Préparation de la table dispo_alim à partir du dataframe aliments sur les bilans alimentaires de la FAO
dispo_alim = aliments
dispo_alim['Disponibilité alimentaire en quantité tonnes'] = dispo_alim['Disponibilité alimentaire en quantité (kg/an)'] / 1000
dispo_alim = dispo_alim.rename(columns={'Pays': 'pays', 'Code Pays': 'code_pays', 'Année': 'annee', 'Produit': 'produit', 'Code Produit': 'code_produit', 'Origine': 'origin', 'Disponibilité alimentaire en quantité tonnes': 'dispo_alim_tonnes', 'Disponibilité alimentaire (Kcal/personne/jour)': 'dispo_alim_kcal_p_j', 'Disponibilité de protéines en quantité (kg/an)': 'dispo_prot', 'Disponibilité de matière grasse en quantité (kg/personne/jour)': 'dispo_mat_gr'})
dispo_alim['dispo_prot'] *= 1000
dispo_alim['dispo_mat_gr'] *= 1000
dispo_alim = dispo_alim[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'origin', 'dispo_alim_tonnes', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr']]

In [212]:
# Alimentation de la bdd
dispo_alim.to_sql('dispo_alim', con, if_exists='append', index=False)

In [214]:
# Vérification
cur.execute(""" SELECT * FROM dispo_alim LIMIT 1 """)
cur.fetchall()

[(1,
  1,
  'Arménie',
  2013,
  'Abats Comestible',
  2736,
  'Animale',
  13664.43,
  14.0,
  2444861250.0000005,
  0.39000000000000007)]

### Question 17
La table appelée equilibre_prod contient pour chaque pays, pour chaque produit, et pour l'année disponible 2013, les quantités suivantes :
- disponibilité intérieure
- aliments pour animaux
- semences
- pertes
- transformés
- nourriture
- autres utilisations

Elle contient ces colonnes :
- pays,
- code_pays, 
- année,
- produit,
- code_produit,
- dispo_int,
- alim_ani,
- semences,
- pertes,
- transfo,
- nourriture,
- autres_utilisations.

Définir une clé primaire pertinente pour la table equilibre_prod.  
Intégrer la table dans la base de données relationnelle.


La clé primaire de la table dispo_alim sera id (clé artificielle).  
En cas d'évolution future, la création de l'id [PK] est une solution qui restera stable

In [215]:
#Création de la table equilibre_prod
cur.execute("""CREATE TABLE equilibre_prod(
            id INTEGER PRIMARY KEY,
            pays TEXT, 
            code_pays INTEGER, 
            annee INTEGER, 
            produit TEXT, 
            code_produit INTEGER, 
            dispo_int REAL, 
            alim_ani REAL, 
            semences REAL, 
            pertes REAL, 
            nourriture REAL, 
            autres_utilisations REAL)""")

In [216]:
#Préparation de la table equilibre_prod à partir du dataframe aliments sur les bilans alimentaires de la FAO
equilibre_prod = aliments
equilibre_prod = equilibre_prod.rename(columns={'Pays': 'pays', 'Code Pays': 'code_pays', 'Année': 'annee', 'Produit': 'produit', 'Code Produit': 'code_produit', 'Disponibilité intérieure': 'dispo_int', 'Aliments pour animaux': 'alim_ani', 'Semences': 'semences', 'Pertes': 'pertes', 'Nourriture': 'nourriture', 'Autres Utilisations': 'autres_utilisations'})
equilibre_prod = equilibre_prod[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'dispo_int', 'alim_ani',  'semences', 'pertes', 'nourriture', 'autres_utilisations']]

In [217]:
# Alimentation de la bdd
equilibre_prod.to_sql('equilibre_prod', con, if_exists='append', index=False)

In [218]:
# Vérification
cur.execute(""" SELECT * FROM equilibre_prod LIMIT 1 """)
cur.fetchall()

[(1,
  'Arménie',
  1,
  2013,
  'Abats Comestible',
  2736,
  17000000.0,
  0.0,
  0.0,
  0.0,
  14000000.0,
  3000000.0)]

### Question 18
La table appelée sous_nutrition, contient le nombre de personnes en sous-alimentation pour chaque pays et pour chaque année.
Elle contient 4 colonnes : 
- pays,
- code_pays,
- année,
- nb_personnes.

Définir une clé primaire pertinente pour la table sous_nutrition.  
Intégrer la table dans la base de données relationnelle.


La clé primaire retenue pour la table sous_nutrition sera le couple pays, année.

In [219]:
#Création de la table sous_nutrition
cur.execute("""CREATE TABLE sous_nutrition(
            pays TEXT, code_pays INTEGER, annee TEXT, nb_personnes REAL, CONSTRAINT pays_annee PRIMARY KEY (pays, annee))""")

In [220]:
# Preparation de la table sous_nutrition
sous_nutri = sous_nutrition_prep
sous_nutri = sous_nutri.rename(columns={'Zone': 'pays', 'Code zone': 'code_pays', 'Année': 'annee', 'Valeur': 'nb_personnes'})
sous_nutri = sous_nutri[['pays', 'code_pays', 'annee', 'nb_personnes']]

In [222]:
# Alimentation de la bdd
sous_nutri.to_sql('sous_nutrition', con, if_exists='append', index=False)

In [223]:
# Vérification
cur.execute(""" SELECT * FROM sous_nutrition LIMIT 3 """)
cur.fetchall()

[('Afghanistan', 2, '2012-2014', 8100000.0),
 ('Afrique du Sud', 202, '2012-2014', 2600000.0),
 ('Albanie', 3, '2012-2014', 200000.0)]

### Question 19
Formuler les requêtes SQL pour répondre aux questions suivantes :

#### Quels sont les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant ?

In [224]:
#Ratio disponibilité alimentaire en protéines (en kg et kcal) par habitant
cur.execute("""
SELECT pays, ROUND(SUM(dispo_prot)/1000, 2) AS dispo_prot_kg, SUM(dispo_alim_kcal_p_j) AS dispo_kcal 
FROM dispo_alim
WHERE annee = 2013
GROUP BY pays 
ORDER BY(dispo_prot_kg) 
DESC LIMIT 10
""")
cur.fetchall()

[('Chine, continentale', 49587018187.75, 3112.0),
 ('Inde', 27380733201.0, 2454.0),
 ("États-Unis d'Amérique", 12782292853.3, 3682.0),
 ('Brésil', 6948283671.3, 3262.0),
 ('Indonésie', 5672707798.0, 2776.0),
 ('Fédération de Russie', 5359938692.1, 3360.0),
 ('Pakistan', 4351259662.75, 2438.0),
 ('Japon', 4067622634.0, 2726.0),
 ('Nigéria', 4041071420.75, 2700.0),
 ('Mexique', 3910550344.4, 3068.0)]

#### Quels sont les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant ?

In [225]:
cur.execute("""
SELECT PAYS, ROUND(SUM(dispo_prot)/1000, 2) AS dispo_prot_kg
FROM dispo_alim
WHERE annee = 2013
GROUP BY pays
ORDER BY(dispo_prot_kg)
ASC LIMIT 10
""").fetchall()

[('Saint-Kitts-et-Nevis', 1425427.2),
 ('Dominique', 2004638.4),
 ('Bermudes', 2143791.0),
 ('Grenade', 2677348.0),
 ('Kiribati', 2730448.2),
 ('Antigua-et-Barbuda', 2742646.5),
 ('Saint-Vincent-et-les Grenadines', 3432251.95),
 ('Sao Tomé-et-Principe', 3740629.5),
 ('Sainte-Lucie', 5695708.2),
 ('Samoa', 5931505.5)]

#### Quelle est la quantité totale (en kg) de produits perdus par pays?

In [226]:
cur.execute("""
SELECT pays, SUM(pertes)
FROM equilibre_prod
WHERE annee = 2013
GROUP BY pays
ORDER BY SUM(pertes)
DESC LIMIT 20
""").fetchall()

[('Chine, continentale', 89575000000.0),
 ('Brésil', 75914000000.0),
 ('Inde', 55930000000.0),
 ('Nigéria', 19854000000.0),
 ('Indonésie', 13081000000.0),
 ('Turquie', 12036000000.0),
 ('Mexique', 8289000000.0),
 ('Égypte', 7608000000.0),
 ('Ghana', 7442000000.0),
 ("États-Unis d'Amérique", 7162000000.0),
 ('Viet Nam', 6743000000.0),
 ('Pakistan', 5897000000.0),
 ('Thaïlande', 5749000000.0),
 ("Iran (République islamique d')", 5450000000.0),
 ('Fédération de Russie', 4997000000.0),
 ('Angola', 4799000000.0),
 ('Ukraine', 4138000000.0),
 ('Bangladesh', 4080000000.0),
 ('Cameroun', 3981000000.0),
 ('Allemagne', 3781000000.0)]

#### Quels sont les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte ?

In [228]:
cur.execute("""
SELECT pays, SUM(nb_personnes) AS personnes
FROM sous_nutrition
GROUP BY pays
ORDER BY personnes DESC LIMIT 10
""").fetchall()

[('Inde', 213100000.0),
 ('Chine, continentale', 136200000.0),
 ('Pakistan', 38400000.0),
 ('Bangladesh', 26400000.0),
 ('Éthiopie', 25500000.0),
 ('Indonésie', 19100000.0),
 ('République-Unie de Tanzanie', 16500000.0),
 ('Philippines', 14000000.0),
 ('Ouganda', 12600000.0),
 ('Nigéria', 12200000.0)]

#### Quels sont les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé ?

In [229]:
cur.execute("""
SELECT produit, ROUND(SUM(autres_utilisations/dispo_int), 2) AS ratio
FROM equilibre_prod
GROUP BY produit
ORDER BY ratio DESC LIMIT 10
""").fetchall()

[('Alcool, non Comestible', 112.0),
 ('Huile de Palme', 96.57),
 ('Huil Plantes Oleif Autr', 85.36),
 ('Graisses Animales Crue', 46.64),
 ('Huile de Palmistes', 44.87),
 ('Huile de Colza&Moutarde', 42.56),
 ('Plantes Aquatiques', 40.51),
 ('Huile de Coco', 35.97),
 ('Manioc', 28.17),
 ('Edulcorants Autres', 23.44)]